<a href="https://colab.research.google.com/github/ayandalab/Sign-Language/blob/main/Sign_Language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1QtuAY57-FVeMiDHExgYN4-kEgGElQYH-',
dest_path='content/asl_alphabets.zip',
unzip=True)

Unzipping...Done.


In [2]:
import pandas as pd                              # Data analysis and manipultion tool
import numpy as np                                 # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf                             # Deep Learning Tool
import os                                             # OS module in Python provides a way of using operating system dependent functionality
import cv2                                             # Library for image processing
from sklearn.model_selection import train_test_split    # For splitting the data into train and validation set

In [3]:
labels = pd.read_csv("/content/content/asl_alphabets/Training_set.csv")              # loading the labels
labels.head()                                                                  # will display the first five rows in labels dataframe

,filename,label
0,Image_1.jpg,B
1,Image_2.jpg,P
2,Image_3.jpg,J
3,Image_4.jpg,T
4,Image_5.jpg,B


In [4]:
labels.tail()                   # will display the last five rows in labels dataframe

,filename,label
60895,Image_60896.jpg,S
60896,Image_60897.jpg,I
60897,Image_60898.jpg,X
60898,Image_60899.jpg,Q
60899,Image_60900.jpg,K


In [5]:
file_paths = [[fname, '/content/content/asl_alphabets/train/' + fname] for fname in labels['filename']]

In [6]:
# Confirm if number of images is same as number of labels given
if len(labels) == len(file_paths):
    print('Number of labels i.e. ', len(labels), 'matches the number of filenames i.e. ', len(file_paths))
else:
    print('Number of labels does not match the number of filenames')

Number of labels i.e.  60900 matches the number of filenames i.e.  60900


In [7]:
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
images.head()

,filename,filepaths
0,Image_1.jpg,/content/content/asl_alphabets/train/Image_1.jpg
1,Image_2.jpg,/content/content/asl_alphabets/train/Image_2.jpg
2,Image_3.jpg,/content/content/asl_alphabets/train/Image_3.jpg
3,Image_4.jpg,/content/content/asl_alphabets/train/Image_4.jpg
4,Image_5.jpg,/content/content/asl_alphabets/train/Image_5.jpg


In [8]:
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')
train_data.head()

,filename,filepaths,label
0,Image_1.jpg,/content/content/asl_alphabets/train/Image_1.jpg,B
1,Image_2.jpg,/content/content/asl_alphabets/train/Image_2.jpg,P
2,Image_3.jpg,/content/content/asl_alphabets/train/Image_3.jpg,J
3,Image_4.jpg,/content/content/asl_alphabets/train/Image_4.jpg,T
4,Image_5.jpg,/content/content/asl_alphabets/train/Image_5.jpg,B


In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['label'])

In [10]:
data = [] # initialize an empty numpy array
image_size = 100 # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):

    img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE) # converting the image to gray scale

    new_img_array = cv2.resize(img_array, (image_size, image_size)) # resizing the image array
    data.append([new_img_array, train_data['label'][i]])

In [11]:
# image pixels of a image
data[5]

[array([[ 31,  29,  29, ...,  29,  28,  29],
        [ 30, 123, 125, ...,  80, 107,  71],
        [ 29, 100, 105, ..., 110, 112,  68],
        ...,
        [ 30, 223, 225, ..., 173, 177, 104],
        [ 30, 222, 224, ..., 175, 176, 102],
        [ 29, 122, 122, ..., 101, 101,  64]], dtype=uint8), 4]

In [12]:
np.random.shuffle(data)

In [13]:
x = []
y = []
for image in data:
    x.append(image[0])
    y.append(image[1])

# converting x & y to numpy array as they are list
x = np.array(x)
y = np.array(y)

In [14]:
np.unique(y, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]),
 array([2100, 2100, 2100, 2100, 2100, 2100, 2100, 2100, 2100, 2100, 2100,
        2100, 2100, 2100, 2100, 2100, 2100, 2100, 2100, 2100, 2100, 2100,
        2100, 2100, 2100, 2100, 2100, 2100, 2100]))

In [15]:
x = x.reshape(-1, 100, 100, 1)

In [16]:
# split the data
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.1, random_state = 42)

In [17]:
cnn = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 1)),
tf.keras.layers.MaxPooling2D((2, 2)),

tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
tf.keras.layers.MaxPooling2D((2, 2)),

# tf.keras.layers.Flatten(input_shape=(100, 100, 1)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(64, activation='relu'),
tf.keras.layers.Dense(32, activation='sigmoid')
])

In [18]:
cnn.compile(optimizer='adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy'])

In [19]:
cnn.fit(X_train, y_train, epochs=2, batch_size=1)

Epoch 1/2
54810/54810 [==============================] - 281s 5ms/step - loss: 2.8780 - accuracy: 0.1804
Epoch 2/2
54810/54810 [==============================] - 249s 5ms/step - loss: 0.8759 - accuracy: 0.7003


In [20]:
cnn.evaluate(X_val, y_val)

191/191 [==============================] - 2s 9ms/step - loss: 0.6358 - accuracy: 0.7770


[0.6358065009117126, 0.777011513710022]

In [21]:
# Loading the order of the image's name that has been provided
test_image_order = pd.read_csv("/content/content/asl_alphabets/Testing_set.csv")
test_image_order.head()

,filename
0,Image_1.jpg
1,Image_2.jpg
2,Image_3.jpg
3,Image_4.jpg
4,Image_5.jpg


In [26]:
file_paths = [[fname, '/content/content/asl_alphabets/test/' + fname] for fname in test_image_order['filename']]

In [23]:
# Confirm if number of images is same as number of labels given
if len(test_image_order) == len(file_paths):
    print('Number of image names i.e. ', len(test_image_order), 'matches the number of file paths i.e. ', len(file_paths))
else:
    print('Number of image names does not match the number of filepaths')

Number of image names i.e.  26100 matches the number of file paths i.e.  26100


In [24]:
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_images.head()

,filename,filepaths
0,Image_1.jpg,/content/content/asl_alphabets/test/Image_1.jpg
1,Image_2.jpg,/content/content/asl_alphabets/test/Image_2.jpg
2,Image_3.jpg,/content/content/asl_alphabets/test/Image_3.jpg
3,Image_4.jpg,/content/content/asl_alphabets/test/Image_4.jpg
4,Image_5.jpg,/content/content/asl_alphabets/test/Image_5.jpg


In [25]:
test_pixel_data = [] # initialize an empty numpy array
image_size = 100 # image size taken is 100 here. one can take other size too
for i in range(len(test_images)):

    img_array = cv2.imread(test_images['filepaths'][i], cv2.IMREAD_GRAYSCALE) # converting the image to gray scale

    new_img_array = cv2.resize(img_array, (image_size, image_size)) # resizing the image array

    test_pixel_data.append(new_img_array)

In [27]:
test_pixel_data = np.array(test_pixel_data)

In [28]:
test_pixel_data = test_pixel_data.reshape(-1, 100, 100, 1)

In [29]:
pred = cnn.predict(test_pixel_data)

In [30]:
# The predicted values are the probabilities value
pred[0]

array([4.21573956e-20, 2.13781272e-18, 1.46712343e-17, 2.51452484e-17,
       9.60515196e-13, 1.69695546e-14, 3.11348967e-11, 1.84804214e-10,
       6.29297725e-10, 7.07682912e-10, 3.35707796e-07, 3.92821050e-07,
       1.74838387e-05, 3.54402036e-05, 1.35317168e-05, 1.02971974e-20,
       1.47787529e-16, 1.40477300e-12, 1.45943641e-14, 5.21486446e-13,
       8.12589666e-12, 1.21115076e-10, 5.47058059e-11, 4.36360170e-10,
       3.28917976e-10, 3.49118068e-08, 3.43328610e-09, 1.36471102e-13,
       5.77966697e-09, 0.00000000e+00, 0.00000000e+00, 1.84357863e-32],
      dtype=float32)

In [32]:
prediction = []
for value in pred:
  prediction.append(np.argmax(value))

In [33]:
predictions = le.inverse_transform(prediction)

In [34]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions}) # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False)

# To download the csv file locally
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>